### 전처리 노트북 1/5: 데이터 Polishing
사용법: 
- config / base.yaml에서 asset_path를 raw 데이터가 있는 경로로 설정
- 학습, 검증 데이터의 이름을 electric_train_raw.csv, electric_test_raw.csv 로 변경
- 각 opt에 대하여 셀을 순서대로 실행

결과물:
- 00_electric_train_polished.csv
- 00_electric_test_polished.csv

In [1]:
# Standard library imports
from pathlib import Path

# Data handling and preprocessing
import pandas as pd
import numpy as np
from tqdm import tqdm
    
from preprocessing_utils import Backup, get_data_paths
asset_path, data_path = get_data_paths()

raw_train_path = Path(asset_path, 'electric_train_raw.csv')
raw_test_path  = Path(asset_path, 'electric_test_raw.csv')

SAVE = True

In [2]:
opt = ['train', 'test'][1]

### train 혹은 test 데이터 선택

In [3]:
data = pd.read_csv(raw_train_path if opt == 'train' else raw_test_path)

data.head()

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN


### 의미없는 column 삭제 및 train, test column이름 통일

In [4]:
if opt == 'train':
    del data['Unnamed: 0']
    data.columns = data.columns.str.replace(f'electric_{opt}.', '')
else:
    data.columns = data.columns.str.replace(f'electric_{opt}.', '')
    # make capital letter as small letter
    data.columns = data.columns.str.lower()
    del data['elect']

### num과 중복되는 stn정보 제거, 및 datetime으로 설정하고 tm변수 제거
del data['stn']

data['datetime'] = pd.to_datetime(data['tm'])
# 1시간 앞으로 시간 설정
data['datetime'] = data['datetime'] - pd.DateOffset(hours=1)
data['year'] = data['datetime'].dt.year

data['date'] = data['datetime'].dt.date
data['month'] = data['datetime'].dt.month

del data['tm']

#data['year'] = (data['datetime'] - pd.DateOffset(hours=1)).dt.year

data['is_dummy'] = 0
data['is_dummy'] = data['is_dummy'].astype(int)

data.head()

,num,hh24,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,datetime,year,date,month,is_dummy
0,4816,1,3.0,68.6,2.9,0.0,-0.1,6,1.0,2023-01-01 00:00:00,2023,2023-01-01,1,0
1,4816,2,3.1,69.4,2.7,0.0,0.3,6,1.0,2023-01-01 01:00:00,2023,2023-01-01,1,0
2,4816,3,3.6,68.3,2.3,0.0,1.2,6,1.0,2023-01-01 02:00:00,2023,2023-01-01,1,0
3,4816,4,4.0,69.2,3.1,0.0,1.1,6,1.0,2023-01-01 03:00:00,2023,2023-01-01,1,0
4,4816,5,4.2,69.5,2.5,0.0,2.0,6,1.0,2023-01-01 04:00:00,2023,2023-01-01,1,0


In [5]:
pre_num      = data['num'].values
pre_datetime = data['datetime'].values

___

### 결측치는 -99.0으로 존재함. 인접한 index column의 평균으로 교체

In [6]:
target_col = ['nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', 'n', 'elec',  'sum_load']
if opt == 'test':
    target_col = target_col[:-3]

# for target col, missing value is represented as value -99.0, so detected with range [-99.001, -98.999].
# for target col we need to replace these values with average value of two neighbor values.

for col in target_col:
    print(f'Number of missing values in {col}: {data[col].between(-99.001, -98.999).sum()}')
    data[col] = data[col].apply(lambda x: np.nan if -99.001 < x < -98.999 else x)

    # outlier value가 있는 경우, 이를 이전 값과 다음 값의 평균으로 대체
    data[col] = data[col].interpolate()

Number of missing values in nph_ta: 0
Number of missing values in nph_hm: 0
Number of missing values in nph_ws_10m: 0
Number of missing values in nph_rn_60m: 0
Number of missing values in nph_ta_chi: 0


### 시간 결측 데이터 보충 (test인 경우 is_dummy 표시)

In [7]:
# group as num and year
grouped = data.groupby(['num', 'year'])

result_dict = {}
for name, group in grouped:

    values = pd.to_datetime(group['datetime'])
    if not values.is_monotonic_increasing:
        # where the datetime is not monotonically increasing
        print('not monotonic', name)

    diffs = values.diff()
    max_diff = diffs.max()
    if max_diff > pd.Timedelta('1h'):

        exceeding_dt1 = values[diffs > pd.Timedelta('1h')]
        exceeding_dt2 = values[diffs.shift(-1) > pd.Timedelta('1h')]
        exceeding_dt3 = values[diffs.shift(-2) > pd.Timedelta('1h')]

        exceeding_diff_indices = diffs[diffs > pd.Timedelta('1h')].index
        print(f'not complete, max diff = {max_diff}', name, end=' | ')

        if exceeding_diff_indices.size > 0:
            new_rows = []
            for idx in exceeding_diff_indices:
                if idx > 0 and diffs.loc[idx] == pd.Timedelta('2h'):

                    new_row  = data.loc[idx - 1].copy()
                    new_row_ = data.loc[idx].copy()
                    
                    new_row['datetime'] = pd.to_datetime(new_row['datetime']) + pd.Timedelta('1h')
                    new_row['date'] = new_row['datetime'].date()

                    new_row['is_dummy'] = 1

                    # for other columns, get the average value
                    for col in data.columns:
                        if col in ['num', 'datetime', 'date', 'is_dummy']:
                            continue
                        dtype = data[col].dtype
                        new_row[col] = (new_row[col] + new_row_[col]) / 2
                        new_row[col] = new_row[col].astype(dtype)

                    new_rows.append(new_row)
        
            new_rows = pd.concat(new_rows, axis=1).T
            group = pd.concat([new_rows, group]).sort_values(by='datetime').reset_index(drop=True)

        print('Indices where diff > 1H:', exceeding_dt1.to_list(), exceeding_dt2.to_list(), exceeding_dt3.to_list(), exceeding_diff_indices.to_list())
        print()

    result_dict[name] = group

not complete, max diff = 0 days 02:00:00 (11412, 2023) | Indices where diff > 1H: [Timestamp('2023-09-06 17:00:00')] [Timestamp('2023-09-06 15:00:00')] [Timestamp('2023-09-06 14:00:00')] [2152168]

not complete, max diff = 0 days 02:00:00 (13350, 2023) | Indices where diff > 1H: [Timestamp('2023-11-15 10:00:00')] [Timestamp('2023-11-15 08:00:00')] [Timestamp('2023-11-15 07:00:00')] [2232680]



In [8]:
# 'num' 열에서 값이 변경되는 위치의 인덱스 찾기
change_indices = data.index[data['num'].shift() != data['num']].tolist()
change_indices = np.asarray(change_indices, dtype=int)
num_order = data['num'].iloc[change_indices].values
print(len(num_order) == len(data['num'].unique()))

# Combine all groups into a single DataFrame
data = pd.concat(result_dict.values()).reset_index(drop=True)

# 'meta_num' 열을 num_order 배열의 순서로 정렬하도록 설정
data['num'] = pd.Categorical(data['num'], categories=num_order, ordered=True)

# 'meta_num'과 'datetime'을 기준으로 정렬
data = data.sort_values(by=['num', 'datetime']).reset_index(drop=True)

data.head()

True


,num,hh24,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,datetime,year,date,month,is_dummy
0,4816,1,3.0,68.6,2.9,0.0,-0.1,6,1.0,2023-01-01 00:00:00,2023,2023-01-01,1,0
1,4816,2,3.1,69.4,2.7,0.0,0.3,6,1.0,2023-01-01 01:00:00,2023,2023-01-01,1,0
2,4816,3,3.6,68.3,2.3,0.0,1.2,6,1.0,2023-01-01 02:00:00,2023,2023-01-01,1,0
3,4816,4,4.0,69.2,3.1,0.0,1.1,6,1.0,2023-01-01 03:00:00,2023,2023-01-01,1,0
4,4816,5,4.2,69.5,2.5,0.0,2.0,6,1.0,2023-01-01 04:00:00,2023,2023-01-01,1,0


In [9]:
num_order

array([ 4816, 16140,  9735,  9884,  9736,  9885, 16143, 16292, 18527,
        5565, 11376, 18528,  4821,  5566, 18529,  5567,  8994, 14805,
       16593, 18232, 18381, 18530, 18679, 18828, 18977, 13614, 18233,
       18382, 18531, 18680, 18829, 18978, 19127, 19276, 13615, 18085,
       18234, 18383, 18532, 18681, 18830, 18979, 19128, 19277, 19426,
       10487, 10934, 11083, 17639, 17937, 18086, 18235, 18384, 18533,
       18682, 18831, 18980, 19129, 19725, 10935, 11084, 11233, 12276,
       16001, 16895, 17938, 18087, 18236, 18385, 18534, 18683, 18832,
       18981, 19279, 10787, 10936, 11085, 11234, 16896, 17343, 17790,
       17939, 18088, 18237, 18386, 18535, 18684, 18833, 18982, 10937,
       11086, 13470, 13619, 16301, 17642, 17791, 17940, 18089, 18238,
       18387, 18536, 18685, 18834, 18983, 16153, 16302, 17196, 17494,
       17643, 17792, 17941, 18090, 18239, 18388, 18537, 18686, 18835,
       18984, 19133, 19282, 19431, 19580, 19878, 13174, 13323, 14366,
       16303, 16452,

In [10]:
Backup.save_with_key(data, f'backup1_{opt}')

get_backup = True
if get_backup:
    data = Backup.copy_data_with_key(f'backup1_{opt}')

### 연도 내 모든 데이터가 있는 경우만 선택 (test의 경우 확인만)

In [11]:
# 각 num별로 각 연도의 모든 datetime이 존재하지 않는 연도 제거
filtered_list = []
pbar = tqdm(data.groupby('num'))
for num, group in pbar:
    for year, year_group in group.groupby('year'):
        # 해당 연도의 모든 시간 생성
        start_time = pd.Timestamp(f'{year}-01-01 00:00:00')
        end_time = pd.Timestamp(f'{year+1}-01-01 00:00:00')
        all_hours = pd.date_range(start=start_time, end=end_time, freq='1h', inclusive = 'left')
        
        # 모든 시간이 데이터프레임에 존재하는지 확인
        if all(all_hours.isin(year_group['datetime'])):
            # year_group을 all_hours로 필터링
            filtered_year_group = year_group[year_group['datetime'].isin(all_hours)]
            if len(filtered_year_group) != len(all_hours):
                print(f'num: {num}, year: {year}:','some hours are missing or duplicated')
            # 필터링된 year_group을 리스트에 추가
            filtered_list.append(filtered_year_group)
        else:
            print(f'num: {num}, year: {year}:','some hours are missing or duplicated')
            print('all hours:', len(all_hours), 'data hours:', len(year_group))

if opt == 'train':
    print('Concatenate dataframes...')
    filtered_data = pd.concat(filtered_list)
    # 'meta_num' 열을 num_order 배열의 순서로 정렬하도록 설정
    filtered_data['num'] = pd.Categorical(filtered_data['num'], categories=num_order, ordered=True)
    data = filtered_data.sort_values(by=['num', 'datetime']).reset_index(drop=True)

/tmp/ipykernel_4193907/964855129.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pbar = tqdm(data.groupby('num'))
  0%|          | 0/323 [00:00<?, ?it/s]

100%|██████████| 323/323 [00:04<00:00, 76.99it/s]


### 백업

In [12]:
Backup.save_with_key(data, f'backup2_{opt}')
get_backup = False
if get_backup:
    data = Backup.copy_data_with_key(f'backup2_{opt}')

In [13]:
assert (pre_num == data[data['is_dummy'] == 0]['num'].values).all()
assert (pre_datetime ==  pd.to_datetime(data[data['is_dummy'] == 0]['datetime'])).all()

if opt == 'train':
    # we treat every interpolated data as valid, so no dummy data
    data['is_dummy'] = 0

data[data['is_dummy'] == 1].shape, data[data['is_dummy'] == 0].shape

((2, 14), (2829478, 14))

In [14]:
cleaned_base_columns = [ 'is_dummy', 'num', 'year', 'datetime', \
                        'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi']

cleaned_train_columns = cleaned_base_columns + ['n','sum_qctr', 'sum_load', 'n_mean_load', 'elec']
# remove 'n'
claned_test_columns   = [col for col in cleaned_base_columns if col != 'n']

if opt == 'train':
    data_save = data[cleaned_train_columns].copy()
else:
    data_save = data[claned_test_columns].copy()

rename_dict = {
    'n': 'units',
    'nph_ta': 'temp',
    'nph_hm': 'humid',
    'nph_ws_10m': 'wind',
    'nph_rn_60m': 'rain',
    'nph_ta_chi': 'tchi',
    'discomfort_index': 'dci'
}

data_save = data_save.rename(columns=rename_dict)

In [15]:
data_save.to_csv(Path(asset_path, f'00_electric_{opt}_polished.csv'), index=False)

In [16]:
data_save.columns

Index(['is_dummy', 'num', 'year', 'datetime', 'temp', 'humid', 'wind', 'rain',
       'tchi'],
      dtype='object')

In [17]:
data_save.head()

,is_dummy,num,year,datetime,temp,humid,wind,rain,tchi
0,0,4816,2023,2023-01-01 00:00:00,3.0,68.6,2.9,0.0,-0.1
1,0,4816,2023,2023-01-01 01:00:00,3.1,69.4,2.7,0.0,0.3
2,0,4816,2023,2023-01-01 02:00:00,3.6,68.3,2.3,0.0,1.2
3,0,4816,2023,2023-01-01 03:00:00,4.0,69.2,3.1,0.0,1.1
4,0,4816,2023,2023-01-01 04:00:00,4.2,69.5,2.5,0.0,2.0


___